In [ ]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from nomad.datamodel.datamodel import (
        EntryArchive,
    )

from nomad.datamodel import EntryMetadata
from nomad.datamodel.datamodel import (
    EntryArchive,
)

In [ ]:
import pandas as pd

from nomad_ikz_fz.schema_packages.mypackage import (
    Feed_rod,
    FzCrystal,
    Resistivity,
)

In [ ]:
df_fr_100 = pd.read_excel(
    'excels/Inventarliste Rohstäbe_20240712.xlsx', sheet_name='100mm'
)
df_fr_100.fillna('', inplace=True)

In [ ]:
df_fr_100  # .columns

In [ ]:
import json


def process_dataframe(df):
    counter = 1

    for index, row in df.iterrows():
        fr = Feed_rod()  # Replace with the correct initialization of your object
        counter += 1
        if row['Seriennummer 100mm'] == 'ohne Nr.':
            fr.name = str(row['Seriennummer 100mm']) + str(counter)
        else:
            fr.name = str(row['Seriennummer 100mm'])

        if row['Länge cm '] != '':
            fr.length = float(row['Länge cm '])

        if row['Angespitzt'] == '+':
            fr.sharpened = True
        else:
            fr.sharpened = False
        if row['Geätzt'] == '+':
            fr.etched = True
        else:
            fr.etched = False

        if row['Bemerkung'] != '':
            fr.description = row['Bemerkung']

        if row['Ort'] != '':
            fr.storage_location = row['Ort']
        fr.diameter = '100 mm'

        archive = EntryArchive(metadata=EntryMetadata())
        fr.normalize(archive, None)

        with open(
            '/home/brueckner/ikz-fz-nomad-plugin/feeds/' + fr.name + '.archive.json',
            'w',
        ) as f:
            json.dump({'data': fr.m_to_dict(with_root_def=True)}, f, indent=2)

    print('Processing complete and files saved.')


# Example usage:
process_dataframe(df_fr_100)

In [ ]:
df_fr_126 = pd.read_excel(
    'excels/Inventarliste Rohstäbe_20240712.xlsx', sheet_name='126-130mm'
)
df_fr_126.fillna('', inplace=True)
df_fr_126

In [ ]:
df_fr_126.Resistivity.unique()

In [ ]:
df_fr_126.columns

In [ ]:


def process_dataframe_126(df):
    counter = 1

    for index, row in df.iterrows():
        fr = Feed_rod()  # Replace with the correct initialization of your object
        counter += 1

        if row['Seriennummer 126-130mm'] == 'ohne Nr.':
            fr.name = str(row['Seriennummer 126-130mm']) + str(counter)
        else:
            fr.name = str(row['Seriennummer 126-130mm'])

        if row['Länge cm '] != '':
            fr.length = float(row['Länge cm '])

        if row['Angespitzt'] == '+':
            fr.sharpened = True
        else:
            fr.sharpened = False

        if row['Geätzt'] == '+':
            fr.etched = True
        else:
            fr.etched = False

        if row['Bemerkung'] != '':
            fr.description = row['Bemerkung']

        if row['Ort'] != '':
            fr.storage_location = row['Ort']
        if row['Resistivity'] == 'High Res ':
            fr.feed_rod_resistivity = 'high resistivity'
        else:
            fr.feed_rod_resistivity = 'standard resistivity'

        # if row["Resistivity"] != '': # and row["Resistivity"] == 'Std Res':
        #     fr.feed_rod_resistivity="standard resistivity"
        # elif row["Resistivity"] != '' and row["Resistivity"] == 'High Res' or 
        # row["Resistivity"] == 'High Res ':
        #     fr.feed_rod_resistivity="high resistivity"
        fr.diameter = '126 - 130 mm'
        archive = EntryArchive(metadata=EntryMetadata())
        fr.normalize(archive, None)

        with open(
            '/home/brueckner/ikz-fz-nomad-plugin/feeds/' + fr.name + '.archive.json',
            'w',
        ) as f:
            json.dump({'data': fr.m_to_dict(with_root_def=True)}, f, indent=2)

    print('Processing complete and files saved.')


# Example usage:
process_dataframe_126(df_fr_126)

In [ ]:
df_crys = pd.read_excel('excels/Kristalle_Inventar_20240712.xlsx')
df_crys.fillna('', inplace=True)
# df_crys
# Initialize empty lists to store resistivity, res_min, and res_max
resistivity = []
res_min = []
res_max = []

for index, row in df_crys.iterrows():
    value = row['Spez. Widerstand ρ [kΩcm]']
    try:
        # Try to convert the value to float
        float_value = float(value)
        resistivity.append(float_value)
        res_min.append(None)  # No min value for single float entries
        res_max.append(None)  # No max value for single float entries
    except ValueError:
        # If conversion fails, it's a string or not a float-convertible value
        if '-' in value:
            # print(value)
            parts = value.split('-')
            try:
                # Convert both parts to float and determine min and max
                part1 = float(parts[0].strip().replace(',', '.'))
                part2 = float(parts[1].strip().replace(',', '.'))
                min_value = min(part1, part2)
                # print(min_value)
                max_value = max(part1, part2)
                # print(max_value)
                res_min.append(min_value)
                res_max.append(max_value)
                resistivity.append(None)  # No single resistivity value for ranges
            except ValueError:
                # If conversion fails, the parts are not valid floats
                resistivity.append(None)
                res_min.append(None)
                res_max.append(None)
        else:
            resistivity.append(None)
            res_min.append(None)
            res_max.append(None)

# At this point, resistivity, res_min, and res_max lists contain the processed values
# You can add them to the DataFrame or use them as needed
df_crys['resistivity'] = resistivity
df_crys['res_min'] = res_min
df_crys['res_max'] = res_max
df_crys.fillna('', inplace=True)
df_crys

In [ ]:
df_crys.columns

In [ ]:
df_crys  # ["L-Typ"].unique()

In [ ]:


def process_dataframe_crys(df):  # noqa: PLR0912
    counter = 1

    for index, row in df.iterrows():
        fr = FzCrystal()  # Replace with the correct initialization of your object
        counter += 1

        if str(row['Versuchsnummer']) != '':
            fr.name = str(row['Versuchsnummer'])

        if str(row['Anlage']) != '':
            fr.fz_furnace = str(row['Anlage'])
            print(fr.fz_furnace)

        if row['Orientierung'] != '':
            fr.orientation = str(row['Orientierung'])

        if (
            row['L-Typ'] != ''
            and row['L-Typ'] == 'n'
            or row['L-Typ'] == 'p'
            or row['L-Typ'] == 'p/n'
        ):
            fr.doping_type = str(row['L-Typ'])
            # undoped add here
        else:
            fr.doping_type = 'undoped'

        if not isinstance(row['Datum'], pd._libs.tslibs.nattype.NaTType):
            fr.process_date = str(row['Datum'])

        fr.resistivity_measurement = Resistivity()

        if row['resistivity'] != '':
            fr.resistivity_measurement.resistivity = float(row['resistivity'])
        if row['res_min'] != '':
            fr.resistivity_measurement.resistivity_minimum = float(row['res_min'])
        if row['res_max'] != '':
            fr.resistivity_measurement.resistivity_maximum = float(row['res_max'])
        # if row["Spez. Widerstand ρ [kΩcm]"] != '':
        #     if '-' in row["Spez. Widerstand ρ [kΩcm]"]:
        #         fr.resistivity=row["Spez. Widerstand ρ [kΩcm]"].split('-')[0]
        # .replace(',', '.')
        #     else: fr.resistivity=row["Spez. Widerstand ρ [kΩcm]"].replace(',', '.')
        # #     fr.orientresistivityation = float(row["Spez. Widerstand ρ [kΩcm]"])

        if row['Length [mm] '] != '':
            fr.length = float(row['Length [mm] '])

        if row['Ist- Ø [mm] '] != '':
            fr.diameter = float(row['Ist- Ø [mm] '])

        if row['Bemerkung'] != '':
            fr.description = str(row['Bemerkung'])

        if row['Location'] != '':
            fr.location = row['Location']

        archive = EntryArchive(metadata=EntryMetadata())
        fr.normalize(archive, None)

        with open(
            '/home/brueckner/ikz-fz-nomad-plugin/crys/' + fr.name + '.archive.json', 'w'
        ) as f:
            json.dump({'data': fr.m_to_dict(with_root_def=True)}, f, indent=2)

    print('Processing complete and files saved.')


# Example usage:
process_dataframe_crys(df_crys)

/home/brueckner/ikz-fz-nomad-plugin/feeds/44318002.archive.json

In [ ]:
type(df_crys.Datum[0]) is pd._libs.tslibs.nattype.NaTType

In [ ]:
fr = FzCrystal()

In [ ]:
archive = EntryArchive(metadata=EntryMetadata())
# fr.normalize(archive, None)

In [ ]:
fr.resistivity_measurement = Resistivity()

In [ ]:
fr.resistivity_measurement.resistivity = 1.0